In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [2]:
from google.colab import files
files.upload()

Saving Loanprdication.csv to Loanprdication.csv


{'Loanprdication.csv': b'Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status\r\nLP001002,Male,No,0,Graduate,No,5849,0,,360,1,Urban,Y\r\nLP001003,Male,Yes,1,Graduate,No,4583,1508,128,360,1,Rural,N\r\nLP001005,Male,Yes,0,Graduate,Yes,3000,0,66,360,1,Urban,Y\r\nLP001006,Male,Yes,0,Not Graduate,No,2583,2358,120,360,1,Urban,Y\r\nLP001008,Male,No,0,Graduate,No,6000,0,141,360,1,Urban,Y\r\nLP001011,Male,Yes,2,Graduate,Yes,5417,4196,267,360,1,Urban,Y\r\nLP001013,Male,Yes,0,Not Graduate,No,2333,1516,95,360,1,Urban,Y\r\nLP001014,Male,Yes,3+,Graduate,No,3036,2504,158,360,0,Semiurban,N\r\nLP001018,Male,Yes,2,Graduate,No,4006,1526,168,360,1,Urban,Y\r\nLP001020,Male,Yes,1,Graduate,No,12841,10968,349,360,1,Semiurban,N\r\nLP001024,Male,Yes,2,Graduate,No,3200,700,70,360,1,Urban,Y\r\nLP001027,Male,Yes,2,Graduate,,2500,1840,109,360,1,Urban,Y\r\nLP001028,Male,Yes,2,Graduate,No,3073,8106,200,360,1,Ur

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Loanprediction').getOrCreate()


**Load DataSet**

In [4]:
df = spark.read.csv('Loanprdication.csv', header=True, sep=',', inferSchema=True)

In [5]:
df.show(5)

+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|              0.0|      NULL|             360|             1|        Urban|          Y|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|          Y

In [6]:
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = true)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: string (nullable = true)



In [7]:
df.dtypes

[('Loan_ID', 'string'),
 ('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'int'),
 ('Loan_Amount_Term', 'int'),
 ('Credit_History', 'int'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

**Data** **Analysis**

In [8]:
# display count based on loan status

df.groupBy('Loan_Status').count().show()

+-----------+-----+
|Loan_Status|count|
+-----------+-----+
|          Y|  422|
|          N|  192|
+-----------+-----+



In [9]:
df.select("Credit_History" ).distinct().show()

+--------------+
|Credit_History|
+--------------+
|             1|
|             0|
|          NULL|
+--------------+



In [10]:
df.select("Credit_History", "Loan_Status" ).groupBy('Loan_Status').agg(F.avg('Credit_History')).show()


+-----------+-------------------+
|Loan_Status|avg(Credit_History)|
+-----------+-------------------+
|          Y| 0.9818181818181818|
|          N| 0.5418994413407822|
+-----------+-------------------+



In [11]:
df.select('Gender', 'Loan_status').groupBy('Loan_status', 'Gender').count().show()

+-----------+------+-----+
|Loan_status|Gender|count|
+-----------+------+-----+
|          N|Female|   37|
|          Y|  NULL|    8|
|          Y|Female|   75|
|          N|  NULL|    5|
|          Y|  Male|  339|
|          N|  Male|  150|
+-----------+------+-----+



In [12]:
#correlation matrix
colums = ('ApplicantIncome','CoapplicantIncome', 'Loan_Amount_Term', 'LoanAmount', 'Credit_History')
corr_df = pd.DataFrame()
for i in colums:
  for j in colums:
    corr_df.loc[i,j] = df.corr(i,j)
corr_df

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,LoanAmount,Credit_History
ApplicantIncome,1.000000,-0.116605,-0.015645,0.538290,0.006986
CoapplicantIncome,-0.116605,1.000000,-0.050321,0.190377,-0.058795
Loan_Amount_Term,-0.015645,-0.050321,1.000000,0.058519,0.050145
LoanAmount,0.538290,0.190377,0.058519,1.000000,-0.032711
Credit_History,0.006986,-0.058795,0.050145,-0.032711,1.000000


In [13]:
columns = ('ApplicantIncome','CoapplicantIncome', 'Loan_Amount_Term', 'LoanAmount', 'Credit_History')
corr_df = pd.DataFrame()
for i in columns:
  corr= []
  for j in colums:
    corr.append(round(df.stat.corr(i,j), 2))
  corr_df = pd.concat([corr_df, pd.Series(corr)], axis=1)
corr_df.columns = columns
corr_df.index = columns
corr_df.insert(0, '', columns)
corr_df.set_index('')

,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,LoanAmount,Credit_History
,,,,,
ApplicantIncome,1.00,-0.12,-0.02,0.54,0.01
CoapplicantIncome,-0.12,1.00,-0.05,0.19,-0.06
Loan_Amount_Term,-0.02,-0.05,1.00,0.06,0.05
LoanAmount,0.54,0.19,0.06,1.00,-0.03
Credit_History,0.01,-0.06,0.05,-0.03,1.00


In [91]:
# convert spark dataframe to pandas
pandas_df = df.toPandas()
pandas_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome
0,LP001002,Male,No,0,Graduate,No,5849,0.0,146,360,1,Urban,1,5849.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,0,6091.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,1,3000.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,1,4941.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,1,6000.0


# **SQL** **Operation**

In [14]:
import pyspark.sql as sparksql

In [15]:
df.createOrReplaceTempView('table')

In [16]:
#display top rows from the table
spark.sql("select * from table limit 5").show()

+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|              0.0|      NULL|             360|             1|        Urban|          Y|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        66|             360|             1|        Urban|          Y

In [17]:
spark.sql('select Loan_ID from table where credit_History=1').show()

+--------+
| Loan_ID|
+--------+
|LP001002|
|LP001003|
|LP001005|
|LP001006|
|LP001008|
|LP001011|
|LP001013|
|LP001018|
|LP001020|
|LP001024|
|LP001027|
|LP001028|
|LP001029|
|LP001030|
|LP001032|
|LP001038|
|LP001041|
|LP001046|
|LP001066|
|LP001068|
+--------+
only showing top 20 rows



**# Data Cleaning**

In [18]:
from pyspark.sql import functions as F

# Count null values in each column
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|      0|    13|      3|        15|        0|           32|              0|                0|        22|              14|            50|            0|          0|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+



In [19]:
# get mean value of column
mean = df.select(F.mean(df['LoanAmount'])).collect()[0] [0]
mean

146.41216216216216

In [20]:
df = df.na.fill(mean, ['LoanAmount'])

In [21]:
df.dtypes

[('Loan_ID', 'string'),
 ('Gender', 'string'),
 ('Married', 'string'),
 ('Dependents', 'string'),
 ('Education', 'string'),
 ('Self_Employed', 'string'),
 ('ApplicantIncome', 'int'),
 ('CoapplicantIncome', 'double'),
 ('LoanAmount', 'int'),
 ('Loan_Amount_Term', 'int'),
 ('Credit_History', 'int'),
 ('Property_Area', 'string'),
 ('Loan_Status', 'string')]

In [22]:
# get made value of column
df.groupBy('Gender').count(). orderBy("count", ascending=False). first()[0]


'Male'

In [23]:
# fill the null values fot all the columns
numerical_cols = ['LoanAmount', 'Loan_Amount_Term']
categorical_cols = ['Gender', 'Married', 'Dependents', 'Credit_History', 'Self_Employed']

In [24]:
for col in numerical_cols:
  mean = df.select(F.mean(df[col])).collect()[0][0]
  df = df.na.fill(mean, [col])

In [25]:
for col in categorical_cols:
  mode = df.groupBy('Gender').count(). orderBy("count", ascending=False). first()[0]
  df = df.na.fill(mode, [col])

In [26]:
# Count null values in each column
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+
|      0|     0|      0|         0|        0|            0|              0|                0|         0|               0|            50|            0|          0|
+-------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+



In [27]:
# create new feature column
df = df.withColumn('TotalIncome', F.col('ApplicantIncome') + F.col('CoapplicantIncome'))
df.show(4)

+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+-----------+
| Loan_ID|Gender|Married|Dependents|   Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+-----------+
|LP001002|  Male|     No|         0|    Graduate|           No|           5849|              0.0|       146|             360|             1|        Urban|          Y|     5849.0|
|LP001003|  Male|    Yes|         1|    Graduate|           No|           4583|           1508.0|       128|             360|             1|        Rural|          N|     6091.0|
|LP001005|  Male|    Yes|         0|    Graduate|          Yes|           3000|              0.0|        

In [28]:
from pyspark.sql import functions as F

# Replace 'Y' with 1 and 'N' with 0 in the 'Loan_Status' column
df = df.withColumn('Loan_Status', F.when(F.col('Loan_Status') == 'Y', 1).otherwise(0))

# Show the updated DataFrame
df.show(5, truncate=False)


+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+-----------+
|Loan_ID |Gender|Married|Dependents|Education   |Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|
+--------+------+-------+----------+------------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+-----------+
|LP001002|Male  |No     |0         |Graduate    |No           |5849           |0.0              |146       |360             |1             |Urban        |1          |5849.0     |
|LP001003|Male  |Yes    |1         |Graduate    |No           |4583           |1508.0           |128       |360             |1             |Rural        |0          |6091.0     |
|LP001005|Male  |Yes    |0         |Graduate    |Yes          |3000           |0.0              |66      

 #**Feature Engineering Step**

In [53]:
from pyspark.ml import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [54]:
df.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Gender: string (nullable = false)
 |-- Married: string (nullable = false)
 |-- Dependents: string (nullable = false)
 |-- Education: string (nullable = true)
 |-- Self_Employed: string (nullable = false)
 |-- ApplicantIncome: integer (nullable = true)
 |-- CoapplicantIncome: double (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- Loan_Amount_Term: integer (nullable = true)
 |-- Credit_History: integer (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Loan_Status: integer (nullable = false)
 |-- TotalIncome: double (nullable = true)



In [55]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

In [56]:
categorical_columns= ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'TotalIncome']

In [57]:
df= df.dropna()

In [58]:
#index the string columns
indexers = [StringIndexer(inputCol=col, outputCol="{0}_index".format(col)) for col in categorical_columns]

In [59]:
from pyspark.ml.feature import OneHotEncoder

encoders = [
    OneHotEncoder(
        dropLast=False,
        inputCol=indexer.getOutputCol(),
        outputCol="{0}_encoded".format(indexer.getOutputCol())
    )
    for indexer in indexers
]


In [60]:
input_columns = [encoder.getOutputCol() for encoder in encoders] + numerical_columns

In [61]:
input_columns

['Gender_index_encoded',
 'Married_index_encoded',
 'Dependents_index_encoded',
 'Education_index_encoded',
 'Self_Employed_index_encoded',
 'Property_Area_index_encoded',
 'ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History',
 'TotalIncome']

In [62]:
# vector the encoded values
assembler = VectorAssembler(inputCols=input_columns, outputCol="feature", handleInvalid="skip")

In [63]:
#create a pipeline to transform the data
pipeline = Pipeline(stages = indexers + encoders + [assembler])

In [64]:
data_model = pipeline.fit(df)

In [66]:
transformed_df = data_model.transform(df)

In [67]:
transformed_df.show(1)

+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+----------------+--------------+-------------+-----------+-----------+------------+-------------+----------------+---------------+-------------------+-------------------+--------------------+---------------------+------------------------+-----------------------+---------------------------+---------------------------+--------------------+
| Loan_ID|Gender|Married|Dependents|Education|Self_Employed|ApplicantIncome|CoapplicantIncome|LoanAmount|Loan_Amount_Term|Credit_History|Property_Area|Loan_Status|TotalIncome|Gender_index|Married_index|Dependents_index|Education_index|Self_Employed_index|Property_Area_index|Gender_index_encoded|Married_index_encoded|Dependents_index_encoded|Education_index_encoded|Self_Employed_index_encoded|Property_Area_index_encoded|             feature|
+--------+------+-------+----------+---------+-------------+---------------+-----------------+----------+-----

In [70]:
# get input features and output columns
transformed_df = transformed_df.select('feature', 'Loan_Status')
transformed_df.show(10)

+--------------------+-----------+
|             feature|Loan_Status|
+--------------------+-----------+
|(24,[0,3,5,10,12,...|          1|
|(24,[0,2,7,10,12,...|          0|
|(24,[0,2,5,10,13,...|          1|
|(24,[0,2,5,11,12,...|          1|
|(24,[0,3,5,10,12,...|          1|
|(24,[0,2,6,10,13,...|          1|
|(24,[0,2,5,11,12,...|          1|
|(24,[0,2,8,10,12,...|          0|
|(24,[0,2,6,10,12,...|          1|
|(24,[0,2,7,10,12,...|          0|
+--------------------+-----------+
only showing top 10 rows



In [71]:
transformed_df.select('Loan_Status').show(10)

+-----------+
|Loan_Status|
+-----------+
|          1|
|          0|
|          1|
|          1|
|          1|
|          1|
|          1|
|          0|
|          1|
|          0|
+-----------+
only showing top 10 rows



In [72]:
#split the data for data train and test
train_data, test_data = transformed_df.randomSplit([0.8, 0.2], seed=42)

In [73]:
# Check the schema and first 5 rows of train_data
train_data.printSchema()
train_data.show(5)

root
 |-- feature: vector (nullable = true)
 |-- Loan_Status: integer (nullable = false)

+--------------------+-----------+
|             feature|Loan_Status|
+--------------------+-----------+
|(24,[0,2,5,10,12,...|          1|
|(24,[0,2,5,10,12,...|          1|
|(24,[0,2,5,10,12,...|          0|
|(24,[0,2,5,10,12,...|          1|
|(24,[0,2,5,10,12,...|          1|
+--------------------+-----------+
only showing top 5 rows



# Model training and **testing**
**bold text**

In [75]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


In [76]:
lr = LogisticRegression(featuresCol='feature', labelCol='Loan_Status')
lr_model = lr.fit(train_data)

In [78]:
# predict on test data
predictions = lr_model.transform(test_data)
predictions.show(5)

+--------------------+-----------+--------------------+--------------------+----------+
|             feature|Loan_Status|       rawPrediction|         probability|prediction|
+--------------------+-----------+--------------------+--------------------+----------+
|(24,[0,2,5,10,12,...|          1|[-2.1846898559126...|[0.10113379480388...|       1.0|
|(24,[0,2,5,10,12,...|          1|[-2.2518690057277...|[0.09518837018877...|       1.0|
|(24,[0,2,5,10,12,...|          1|[-2.1397712159779...|[0.10529094011977...|       1.0|
|(24,[0,2,5,10,12,...|          1|[-2.1946068975708...|[0.10023583798354...|       1.0|
|(24,[0,2,5,10,12,...|          1|[-2.3207487209725...|[0.08941907725310...|       1.0|
+--------------------+-----------+--------------------+--------------------+----------+
only showing top 5 rows



In [82]:
predictions = lr_model.transform(test_data)
auc = BinaryClassificationEvaluator(labelCol='Loan_Status').evaluate(predictions)
print("AUC:", auc)

AUC: 0.8194444444444444


In [83]:
rf = RandomForestClassifier(featuresCol='feature', labelCol='Loan_Status')
rf_model = rf.fit(train_data)

In [84]:
predictions = rf_model.transform(test_data)
auc = BinaryClassificationEvaluator(labelCol='Loan_Status').evaluate(predictions)
print("AUC:", auc)

AUC: 0.8009259259259259


In [85]:
auc

0.8009259259259259

In [89]:
auc_percentage = auc * 100
print(auc_percentage)

80.0925925925926
